# Cebuano Stemmer Demo

`Accepts a Cebuano word and returns the root word and its affixes`

In [10]:
from stemmer import stem_word

ceb_word = raw_input('Input a Cebuano word: ')
word = stem_word(word=ceb_word)

print(word.print_stem_results())

Input a Cebuano word: dagkong
dagkong:[(dagko), {-,-,ng}]


# Cebuano POS Tagger Demo

`Accepts a Cebuano sentence and return POS tagged sentence`

In [3]:
from tagger import tag_sentence

ceb_sentence = raw_input('Input a Cebuano sentence: ')
words = tag_sentence(text=ceb_sentence)

sentence = ''
for word in words:
    sentence += str(word) + ' '

print(sentence)

Input a Cebuano sentence: Patay ang usa ka security guard nga hubog nagmaniho sa iyang motorsiklo ug  nakabangga sa gikasugat nga laing motorsiklo atol sa Pasko sa national highway sa Barangay Fuente, lungsod sa Carmen.
Patay/NOUN ang/PART usa/NOUN ka/PART security/NOUN guard/NOUN nga/PART hubog/ADJ nagmaniho/VERB sa/PART iyang/PRON motorsiklo/NOUN ug/CONJ nakabangga/VERB sa/PART gikasugat/OTH nga/PART laing/ADJ motorsiklo/NOUN atol/PART sa/PART Pasko/NOUN sa/PART national/VERB highway/NOUN sa/PART Barangay/NOUN Fuente/NOUN ,/SYM lungsod/NOUN sa/PART Carmen/NOUN ./SYM 


# Stemmer Evaluation


`Evaluates the stemmer by getting the number of predicted root words vs. actual root words`

* The input words are already tokenized and stemmed

In [1]:
import stemmer_evaluator as se
import pandas as pd

result = se.to_panda_data()

df = pd.DataFrame(result['data'], index=result['index'])
df_valid = df[(df.is_valid == True)]
valid = df_valid.count()['is_valid']
print('Total tokens: ' + str(df.count()['is_valid']))
print('Correct root words: ' + str(df[(df.is_valid == True)].count()['is_valid']))
print('Incorrect root words: ' + str(df[(df.is_valid == False)].count()['is_valid']))
print("Correct / incorrect root percentage: " + str((valid / float(df.count()['is_valid'])) * 100))
print('\n')
print('Dictionary entry tokens: ' + str(df[(df.is_entry == True)].count()['is_entry']))
print('Correct dictionary entry tokens: ' + str(df[(df.is_entry == True) & (df.is_valid == True)].count()['is_entry']))
print('Incorrect dictionary entry tokens: ' + str(df[(df.is_entry == True) & (df.is_valid == False)].count()['is_entry']))
print('\n')
print('Non-dictionary entry tokens: ' + str(df[(df.is_entry == False)].count()['is_entry']))
print('Correct non-dictionary entry tokens: ' + str(df[(df.is_entry == False) & (df.is_valid == True)].count()['is_entry']))
print('Incorrect non-dictionary entry tokens: ' + str(df[(df.is_entry == False) & (df.is_valid == False)].count()['is_entry']))
print('\n')

df.tail(n=10)


Total tokens: 1298
Correct root words: 955
Incorrect root words: 343
Correct / incorrect root percentage: 73.57473035439138


Dictionary entry tokens: 974
Correct dictionary entry tokens: 878
Incorrect dictionary entry tokens: 96


Non-dictionary entry tokens: 324
Correct non-dictionary entry tokens: 77
Incorrect non-dictionary entry tokens: 247




,infix,is_entry,is_root,is_valid,prefix,root,suffix
nipahibawo,NaN,False,False,False,ni,pahibaw,o
naghuwat,NaN,True,False,True,nag,huwat,NaN
unsaon,NaN,True,False,True,NaN,unsa,on
mamahimong,NaN,True,False,True,ma,himo,ng
syudad,NaN,True,True,True,NaN,syudad,NaN
sirhan,NaN,True,False,False,NaN,sir,han
kalapasan,NaN,True,False,True,ka,lapas,an
bawian,NaN,False,False,False,ba,wi,an
nakalapas,NaN,True,False,True,naka,lapas,NaN
nagkadaiyang,NaN,True,False,False,nag,kadaiya,ng


# Tagger Evaluation


`Evaluates the tagger by getting the number of predicted pos tags vs. actual pos tags (F-score)`

In [3]:
from pandas_ml import ConfusionMatrix
from sklearn.metrics import f1_score
import tagger_evaluator as te

words = te.tag_test_sentences()
y_true = te.extract_actual_pos_tags()
y_pred = te.extract_predicted_pos_tags(words=words)
confusion_matrix = ConfusionMatrix(y_true, y_pred)
# confusion_matrix.print_stats()
print(confusion_matrix)

print('\nOverall F-Score\n')
f1score = f1_score(y_true, y_pred, average='micro')
print(f1score)
print('\n')

df = f1_score(y_true, y_pred, average=None)
print('Individual F-Score' + '\n')
print('ADJ  : ' + str(df[0]))
print('ADV  : ' + str(df[1]))
print('CONJ : ' + str(df[2]))
print('DET  : ' + str(df[3]))
print('NOUN : ' + str(df[4]))
print('NUM  : ' + str(df[5]))
print('OTH  : ' + str(df[6]))
print('PART : ' + str(df[7]))
print('PRON : ' + str(df[8]))
print('SYM  : ' + str(df[9]))
print('VERB : ' + str(df[10]))

f_score = 0
for i in range(0, 11):
    if i != 6:
        f_score += df[i]

print(f_score / 10.0)

Number of tokens: 236

Disambiguated: 100.0% (236) 

Predicted  ADJ  ADV  CONJ  DET  NOUN  NUM  OTH  PART  PRON  SYM  VERB  __all__
Actual                                                                        
ADJ          2    1     0    0     0    0    1     0     0    0     0        4
ADV          0    1     2    0     1    0    1     0     0    0     0        5
CONJ         0    0     4    0     0    0    0     0     0    0     0        4
DET          0    0     9   31     0    0    0     1     0    0     0       41
NOUN         6    0     0    0    43    0    2     0     3    0     4       58
NUM          0    0     0    0     0    3    0     0     0    0     0        3
OTH          0    0     0    0     0    0    0     0     0    0     0        0
PART         0    4     2    0     0    0    0    17     2    0     0       25
PRON         0    0     0    0     1    0    0     0    26    0     0       27
SYM          0    0     0    0     0    0    0     0     0   37     0       37